In [ ]:
# Abrir o google, abrir e logar no site do maktazap. Informar a planilha com os números dos alunos e enviar a mensagem.
#Bibliotecas
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import os
import datetime as dt
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

In [ ]:
#Base!
data = pd.read_excel(r'C:\Users\gugag\01.Scripts\Alunos-EaD-Loop.xlsx')

data.head()

# Cria um clone do DataFrame original
data_copy = data.copy()

# Converte a coluna 'Telefone' para string
data_copy['Telefone'] = data_copy['Telefone'].astype(str)
#data_copy.head()

In [ ]:
driver = webdriver.Chrome()

driver.get('http://google.com.br')

chrome_options = Options()
chrome_options.add_argument("--disable-notifications")

In [ ]:
#Abrindo a pagina especifica
driver.get(r'https://app3.mktzap.com.br/login')
time.sleep(1)
driver.maximize_window()

### Login

In [ ]:
# Clicando no botão de login
#driver.find_element(By.XPATH, '//*[@id="responsive-navbar-nav"]/div/div[4]/div[1]/button').click()
#time.sleep(1)
#Alterando para a nova janela aberta
# Pega todos os identificadores de janelas abertas
windows = driver.window_handles

# Alterna para a nova janela (a última aberta)
#driver.switch_to.window(windows[-1])
#Clicando no email.
driver.find_element(By.XPATH, '//*[@id="login_fldLoginUserName"]').click()
# Digitando o email
driver.find_element(By.XPATH, '//*[@id="login_fldLoginUserName"]').send_keys("#######")
#Alternado o campo - Campo de senha
driver.find_element(By.XPATH, '//*[@id="login_fldLoginPassword"]').click()
driver.find_element(By.XPATH, '//*[@id="login_fldLoginPassword"]').send_keys("####")
time.sleep(1)
#Clicando para entrar
driver.find_element(By.XPATH, '//*[@id="login_btnLogin"]').click()
#Aguardando login
time.sleep(5)

### Loop com try e exportação

In [ ]:
# Criando uma lista para armazenar os resultados
resultados = []

# Loop para iterar sobre os telefones no DataFrame
for i, telefone in data_copy['Telefone'].items():
    try:
        # Aperta para enviar mensagem
        while len(driver.find_elements(By.XPATH, '//*[@id="dropdownMenuLink"]')) <1:
            time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="dropdownMenuLink"]').click()
        
        # Clicando em Mensagem no WhatsApp
        driver.find_element(By.XPATH, '/html/body/div[1]/section[1]/div[1]/div[2]/div/div/div[2]/div[1]/div').click()
        time.sleep(2)
        # Selecionando Unisa Acadêmico
        while len(driver.find_elements(By.XPATH, '/html/body/div[1]/section[1]/div[1]/div[2]/div/div/div[2]/div[1]/div')) < 1:
            time.sleep(1)  # Espera 1 segundo antes de verificar novamente
            driver.find_element(By.XPATH, '//*[@id="activeChannel"]/optgroup/option').click()
        driver.find_element(By.XPATH, '//*[@id="activeChannel"]/optgroup/option').click()
        time.sleep(1)
        
        # Responsável pelo atendimento
        driver.find_element(By.XPATH, '//*[@id="formActiveWhatsapp"]/div[2]/div/div[2]/div[1]/div/label').click()
        
        # Definindo o DDI
        driver.find_element(By.XPATH, '//*[@id="activeDDI"]').click()
        driver.find_element(By.XPATH, '//*[@id="activeDDI"]').clear()
        driver.find_element(By.XPATH, '//*[@id="activeDDI"]').send_keys('55')
        
        # Preenchendo o telefone
        driver.find_element(By.XPATH, '//*[@id="formActiveWhatsapp"]/div[2]/div/div[3]/div/div[2]/input').click()
        driver.find_element(By.XPATH, '//*[@id="formActiveWhatsapp"]/div[2]/div/div[3]/div/div[2]/input').send_keys(telefone)
        
        # Selecionando o modelo de mensagem
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="activeHsmModel"]').click()
        time.sleep(0.5)
        driver.find_element(By.XPATH, '//*[@id="activeHsmModel"]/option[34]').click()
        
        # Clicando fora para preparar o envio
        time.sleep(0.5)
        driver.find_element(By.XPATH, '//*[@id="activeHsmModel"]').click()
        
        # Enviando a mensagem
        driver.find_element(By.XPATH, '//*[@id="formActiveWhatsapp"]/div[3]/button[2]').click()
        
        # Espera um pouco para garantir que a mensagem foi enviada
        time.sleep(1.5)

        # Adiciona o telefone com sucesso
        resultados.append({'Telefone': telefone, 'Status': 'Sucesso'})
        
    except Exception as e:
        # Caso ocorra algum erro, adiciona o telefone com falha
        print(f"Erro ao processar o telefone {telefone}: {e}")
        resultados.append({'Telefone': telefone, 'Status': 'Falha'})

driver.quit()

#### Gerando o arquivo

In [ ]:
# Obtendo a data e a hora atual com formato de 24 horas
data_hora_atual = datetime.now().strftime("Dia-%d-%m-Horario-%H-%M")  # Ajustado para incluir "Dia" e "Horário"

# Nome do arquivo com a data e hora (certificando-se de que a extensão seja .xlsx)
nome_arquivo = f"Whats-Enviados-{data_hora_atual}.xlsx"

# Caminho para salvar o arquivo
caminho_arquivo = f"C:\\Users\\gugag\\01.Scripts\\Resultados\\{nome_arquivo}"

# Criando um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)

# Exportando o DataFrame para um arquivo Excel
df_resultados.to_excel(caminho_arquivo, index=False)

print(f"Processamento concluído! O arquivo de resultados foi salvo como {nome_arquivo}.")